### **INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**DOWNLOADING LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [3]:
#@ DOWNLOADING THE LIBRARIES AND DEPENDENCIES:
# !pip install -U d2l
from d2l import torch as d2l

import os, json, multiprocessing
import torch     
from torch import nn                                
from torch.nn import functional as F

### **LOADING PRETRAINED BERT:**
- The original BERT Model has hundreds of millions of parameters. Here, I have installed two versions of pretrained BERT: BERT Base is the original BERT Model that requires a lot computational resources to fine tune while BERT Small is a small version to facilitate demonstration. 

In [4]:
#@ INSTALLING BERT MODEL: 
d2l.DATA_HUB["bert.base"] = (d2l.DATA_URL + 'bert.base.torch.zip',
                             '225d66f04cae318b841a13d32af3acc165f253ac')          # Getting BERT Base Model. 
d2l.DATA_HUB["bert.small"] = (d2l.DATA_URL + 'bert.small.torch.zip',
                              'c72329e68a732bef0452e4b96a1c341c8910f81f')         # Getting BERT Small Model. 

- The pretrained BERT Model contains a vocab json file that defines the vocabulary set and a pretrained params file of the pretrained parameters. I will define a function to load pretrained BERT parameters. 

In [5]:
#@ LOADING PRETRAINED BERT MODEL PARAMETERS: 
def load_pretrained_model(pretrained_model, num_hiddens, ffn_num_hiddens, 
                          num_heads, num_layers, dropout, max_len, devices):             # Loading Pretrained Model. 
  data_dir = d2l.download_extract(pretrained_model)                                      # Downloading and Extracting Model. 
  vocab = d2l.Vocab()                                                                    # Initializing Vocabulary. 
  vocab.idx_to_token = json.load(open(os.path.join(data_dir, "vocab.json")))             # Initializing BERT Vocabulary. 
  vocab.token_to_idx = {token: idx for idx, token in 
                        enumerate(vocab.idx_to_token)}
  bert = d2l.BERTModel(len(vocab), num_hiddens, norm_shape=[256], 
                       ffn_num_input=256, ffn_num_hiddens=ffn_num_hiddens, 
                       num_heads=4, num_layers=2, dropout=0.2, max_len=max_len, 
                       key_size=256, query_size=256, value_size=256, 
                       hid_in_features=256,mlm_in_features=256,nsp_in_features=256)      # Initializing BERT Model. 
  bert.load_state_dict(torch.load(os.path.join(data_dir, "pretrained.params")))          # Loading Pretrained Model Parameters. 
  return bert, vocab

In [8]:
#@ LOADING PRETRAINED BERT MODEL: 
devices = d2l.try_all_gpus()                                                              # Initialization. 
bert, vocab = load_pretrained_model("bert.small", num_hiddens=256, ffn_num_hiddens=512, 
                                    num_heads=4, num_layers=2, dropout=0.1, max_len=512, 
                                    devices=devices)                                      # Implementation of Function. 